In [5]:
import re
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import (
    WikipediaQueryRun,
    DuckDuckGoSearchRun,
    DuckDuckGoSearchResults,
)
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
ddg = DuckDuckGoSearchResults(
    backend="auto",
    description="A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query. Output is a JSON string array of the query results",
)

query = "Research about the XZ backdoor"
wikipedia_results = wikipedia.run(query)
ddg_results = ddg.run(query)


# 정규 표현식으로 'link' 찾기
links = re.findall(r"link:\s*(https?://[^\]]+)", ddg_results)

# print("wikipedia_results", wikipedia_results)

loader = WebBaseLoader(links)
docs = loader.load()

with open("./research_results.txt", "w") as f:
    for doc in docs:
        final_doc = doc.page_content.replace("\n", " ").replace("\xa0", " ")
        f.write(final_doc)


agent = initialize_agent(
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    tools=[
        wikipedia,
        ddg,
    ],
)

# The Wikipedia results are: {wikipedia_results}.
agent.invoke(f"The DuckDuckGo results are: {ddg_results}.")



> Entering new AgentExecutor chain...
I need to gather more information about the XZ backdoor, specifically its implications and how it operates. The DuckDuckGo results provide several snippets that mention the backdoor's functionality and its impact on the XZ utility. 

Action: DuckDuckGo Results JSON  
Action Input: "XZ backdoor details"  
Observation: [snippet: The XZ Backdoor: Everything You Need to Know Details are starting to emerge about a stunning supply chain attack that sent the open source software community reeling. Illustration: da-kuk/Getty Images, title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: Now that the XZ backdoor has control over sshd, attackers can possess the encryption key used to make the SSH connections hide their malicious code in the SSH login certificate and use it as an entry point to execute various malicious actions on the compromised device - such as st

{'input': "The DuckDuckGo results are: [snippet: Recently, a backdoor was discovered in XZ, a popular library for lossless data compression. Initial research efforts were predominantly concentrated on unpacking the well-disguised attack vector, while the social aspects of the attack received only murmurings. To investigate this attack, I never read a line of code., title: Everything I Know About the XZ Backdoor - Boehs, link: https://boehs.org/node/everything-i-know-about-the-xz-backdoor], [snippet: What does the backdoor do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: What we know about the xz Utils backdoor that almost infected the world, link: https://arstechnica.com/security/2024/04/what-we-know-about-the-xz-utils-backdoor-that-almost-infected-the-world/], [snippet: 2024-02-24: Jia Tan tags and builds v5.6.0 and publishes an xz-5.6..tar.gz distri